Before initializing, Need to Under stand the Data

In [17]:
import pandas as pd
df = pd.read_excel("traffic_stops.xlsx")
print("Top 5 Rows")
print(df.head())
print("\n Columns Info:")
print(df.info())

Top 5 Rows
   stop_date stop_time country_name driver_gender  driver_age_raw  driver_age  \
0 2020-01-01  00:00:00       Canada             M              59          19   
1 2020-01-01  00:01:00        India             M              35          58   
2 2020-01-01  00:02:00          USA             M              26          76   
3 2020-01-01  00:03:00       Canada             M              26          76   
4 2020-01-01  00:04:00       Canada             M              62          75   

  driver_race     violation_raw violation  search_conducted     search_type  \
0       Asian     Drunk Driving  Speeding              True  Vehicle Search   
1       Other             Other     Other             False  Vehicle Search   
2       Black  Signal Violation  Speeding             False           Frisk   
3       Black          Speeding       DUI              True           Frisk   
4       Other          Speeding     Other             False  Vehicle Search   

  stop_outcome  is_arrested

In [41]:
df.columns

Index(['stop_date', 'stop_time', 'country_name', 'driver_gender',
       'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw',
       'violation', 'search_conducted', 'search_type', 'stop_outcome',
       'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number'],
      dtype='object')

To Create a Database

In [ ]:
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password = "SQL@123sql",
)
cursor = connection.cursor()

query = "CREATE DATABASE securecheck"
cursor.execute(query)

To create Tables in the DB

In [12]:
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password = "SQL@123sql",
    database="securecheck"
)
cursor = connection.cursor()

query = "CREATE TABLE checkpost_data (stop_date DATE, stop_time TIME, country_name VARCHAR(100), driver_gender VARCHAR(10), driver_age_raw INT, driver_age INT, driver_race VARCHAR(50), violation_raw VARCHAR(100), violation VARCHAR(100), search_conducted BOOLEAN, search_type VARCHAR(100), stop_outcome VARCHAR(100), is_arrested BOOLEAN, stop_duration VARCHAR(50), drugs_related_stop BOOLEAN,vehicle_number VARCHAR(50))"
cursor.execute(query)

Database and Table is created. Now we need to go back to the data clean and prepare it to push it to the database

Initializing, Cleaning and Preparation of Data

Checking for the NULL Values

In [26]:
df = pd.read_excel("traffic_stops.xlsx")
print("\nMissing values per column:")
print(df.isnull().sum())



Missing values per column:
stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64


Observation : Out of all the columns only search_type has missing values or Null Values

Next - Further Inspection needed if Search is conducted or Not. To fill the Null Value Fields

In [30]:
inconsistent_rows = df[(df['search_type'].isnull()) & (df['search_conducted'] == True)]
print(f"Number of inconsistent rows where is search is conducted: {len(inconsistent_rows)}")

inconsistent_rows2 = df[(df['search_type'].isnull()) & (df['search_conducted'] == False)]
print(f"Number of inconsistent rows where is search is not conducted: {len(inconsistent_rows2)}")

Number of inconsistent rows where is search is conducted: 10870
Number of inconsistent rows where is search is not conducted: 10850


Observation : Almost Half of the Search type data is not filled for both the search scenario

For Null Value where search is conducted, It can be filled with "Search Type Not Specified"

For Null Value where search is not conducted, Need to check values where search conducted is False. To make the data redundant

In [35]:
Labels_used = df[df['search_conducted'] == False]['search_type'].unique()
print(f"The Labels used are : {Labels_used}")

The Labels used are : ['Vehicle Search' 'Frisk' nan]


Observation : It has the Values - 'Vehicle Search' & 'Frisk', which means there is discrpancy in the search_conducted columns.

Next : The search_conducted columns values should be corrected where the Data Points in search_type column is not null

In [ ]:
df.loc[(df['search_type'] == "Vehicle Search") | (df['search_type'] == "Frisk"), 'search_conducted'] = True

Once the Values have been corrected, I can update the Null Values

In [38]:
df.loc[(df['search_type'].isnull()) & (df['search_conducted'] == True), 'search_type'] = "Search Type Not Specified"
df.loc[(df['search_type'].isnull()) & (df['search_conducted'] == False), 'search_type'] = "No Search, Vehicle all green"

Doing a Double check to know if there still some Null Values left

In [39]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64


Oservation : No Null Values now. Data is ready to be pushed to Database.

In [40]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="SQL@123sql",
    database="securecheck"
)
cursor = connection.cursor()

columns = ",".join(df.columns)
placeholders = ",".join(["%s"] * len(df.columns))

insert_query = f"INSERT INTO checkpost_data ({columns}) VALUES ({placeholders})"
data = [tuple(row) for row in df.itertuples(index=False, name=None)]
cursor.executemany(insert_query, data)
connection.commit()


Data Cleaned, Prepared and Pushed to the Database